In [61]:
import json
import numpy as np
with open('./concat.json','r',encoding='utf8')as fp: #824 100
    js = json.load(fp)

In [62]:
def findLineIndex(myjs,cnn_index):
    line_index=[]
    for index,feature in enumerate(myjs["features"]):
        if(feature["pop"]>0):
            continue
        if(cnn_index in feature["start"]["xy"]):
            line_index.append(index)
    return line_index

In [63]:
from math import sqrt
def calDis(point1,point2):
    dis=sqrt((point1[0]-point2[0])**2+(point1[1]-point2[1])**2)
    return dis
def checkConn(line1,line2,th=5):
    if(len(line1)==0 or len(line2)==0): return []
    head1=line1[0]
    bottom1=line1[-1]
    head2=line2[0]
    bottom2=line2[-1]
    if(calDis(head1,head2)<th):
        line1.reverse()
        return line1+line2
    if(calDis(head1,bottom2)<th):
        line1.reverse()
        line2.reverse()
        return line1+line2
    if(calDis(bottom1,head2)<th): 
        return line1+line2
    if(calDis(bottom1,bottom2)<th): 
        line2.reverse()
        return line1+line2
    return []

In [64]:
def connect(myjs,index):
    new_cnn_list=[]
    for cnn_index in myjs["features"][index]["start"]["connect"]:
        line_list=findLineIndex(myjs,cnn_index)
        for line_index in line_list:
            tmp=checkConn(myjs["features"][index]["geometry"]["coordinates"],myjs["features"][line_index]["geometry"]["coordinates"])
            if(len(tmp)==0):
                continue
            myjs["features"][index]["geometry"]["coordinates"]=tmp
            myjs["features"][index]["start"]["xy"]+=myjs["features"][line_index]["start"]["xy"]
            new_cnn_list=new_cnn_list+myjs["features"][line_index]["start"]["connect"]
            myjs["features"][line_index]["pop"]=2
    myjs["features"][index]["start"]["connect"]=new_cnn_list
    if(len(new_cnn_list)==0):
        return 0
    connect(myjs,index)

In [65]:
exclude=[]
new_js=js.copy()
for index,feature in enumerate(new_js["features"]):
    if(feature["pop"]>0):
        continue
    new_js["features"][index]["pop"]=1
    connect(new_js,index)

In [66]:
index=0
while(True):
    if(new_js["features"][index]["pop"]==2):
        new_js["features"].pop(index)
    else:
        index+=1
    if(len(new_js["features"])==index):
        break

In [67]:
filename="connect.json"
with open(filename,'w') as file_obj:
    json.dump(new_js,file_obj)